In [1]:
import porter
import TextRepresenter
from collections import Counter
import math
import re

# EXERCICE 1

In [2]:
blocked_list = ['the', 'a', 'an', 'on', 'behind', 'under', 'there', 'in', 'on']
doc1 = "the new home has been saled on top forecasts"
doc2 = "the home sales rise in july"
doc3 = "there is an increase in home sales in july"
doc4 = "july encounter a new home sales rise"
corpus = [doc1,doc2,doc3,doc4]
N = len(corpus)

## Question 1.1

In [3]:
def normalize_document(d):
    cnt = Counter()
    
    for mot in d.split(' '):
        mot = porter.stem(mot.lower())
        if (mot in blocked_list):
            continue
        cnt[mot] += 1
        
    return dict(cnt)

In [4]:
normalize_document(doc1)

{'been': 1, 'forecast': 1, 'ha': 1, 'home': 1, 'new': 1, 'sale': 1, 'top': 1}

## Question 1.2

In [5]:
#fichier index
def index_corpus(c):
    res = dict()
    i = 0
    for d in c:
        res[i] = normalize_document(d)
        i += 1
    return res

In [6]:
idx = index_corpus(corpus)
idx

{0: {'been': 1,
  'forecast': 1,
  'ha': 1,
  'home': 1,
  'new': 1,
  'sale': 1,
  'top': 1},
 1: {'home': 1, 'juli': 1, 'rise': 1, 'sale': 1},
 2: {'home': 1, 'increas': 1, 'is': 1, 'juli': 1, 'sale': 1},
 3: {'encount': 1, 'home': 1, 'juli': 1, 'new': 1, 'rise': 1, 'sale': 1}}

In [16]:
#fichier index inverse
def inverse_index_corpus(index):
    res = dict()
    #on fait l'ensemble de tous les mots différents présents dans le corpus
    #pour chaque mot on regarde les documents où il est présent
    for doc,mots in index.items():
        for mot,occ in mots.items():
            if mot not in res:
                res[mot]={doc:occ}
            else :
                res[mot][doc] = occ
    return res

In [18]:
inverse_index_corpus(idx)

{'been': {0: 1},
 'encount': {3: 1},
 'forecast': {0: 1},
 'ha': {0: 1},
 'home': {0: 1, 1: 1, 2: 1, 3: 1},
 'increas': {2: 1},
 'is': {2: 1},
 'juli': {1: 1, 2: 1, 3: 1},
 'new': {0: 1, 3: 1},
 'rise': {1: 1, 3: 1},
 'sale': {0: 1, 1: 1, 2: 1, 3: 1},
 'top': {0: 1}}

## Question 1.3

In [27]:
def tf_idf(c):
    tfidf = dict()
    index = index_corpus(c)
    indexinv = inverse_index_corpus(index)
    #calcul du tfidf
    for doc, occurs in index.items():
        tfidf[doc] = dict()
        for mot, nb in occurs.items():
            tf = nb
            df = len(indexinv[mot])
            idf = math.log((1+N)/(1 + df))
            tfidf[doc][mot] = tf*idf
    return tfidf

In [35]:
tfidfindex = tf_idf(corpus)

In [36]:
inverse_index_corpus(tfidfindex)

{'been': {0: 0.9162907318741551},
 'encount': {3: 0.9162907318741551},
 'forecast': {0: 0.9162907318741551},
 'ha': {0: 0.9162907318741551},
 'home': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0},
 'increas': {2: 0.9162907318741551},
 'is': {2: 0.9162907318741551},
 'juli': {1: 0.22314355131420976,
  2: 0.22314355131420976,
  3: 0.22314355131420976},
 'new': {0: 0.5108256237659907, 3: 0.5108256237659907},
 'rise': {1: 0.5108256237659907, 3: 0.5108256237659907},
 'sale': {0: 0.0, 1: 0.0, 2: 0.0, 3: 0.0},
 'top': {0: 0.9162907318741551}}

# EXERCICE 2

## Question 2.1

In [29]:
def buildDocCollectionSimple(fichier):
    balises=[".B",".A",".N",".X",".K",".W"]
    f = open(fichier, "r")
    tab = []
    check = False
    line = f.readline()
    while(line):
        if line[0:2]==".I":
            tab.append([line[3:-1],""])
            check = False
        else:
            if not check and line[0:2]==".T" :
                check = True
                tab[-1][1] += line[3:]
            elif check and line[0:2] not in balises:
                tab[-1][1] += line
            if line[0:2] in balises:
                check = False
        line = f.readline()
    return [(int(tab[i][0]),re.sub(r"^[\n ]*","",tab[i][1].replace('\n',' '))[:-1]) for i in range(len(tab))]

In [30]:
buildDocCollectionSimple("cacmShort-good.txt")

[(1, 'Preliminary Report-International Algebraic Language jgkfldjgfkld'),
 (2, 'Extraction of Roots by Repeated Subtractions for Digital Computers'),
 (3, 'Techniques Department on Matrix Program Schemes'),
 (4, 'Glossary of Computer Engineering and Programming Terminology'),
 (5, 'Two Square-Root Approximations'),
 (6, 'The Use of Computers in Inspection Procedures'),
 (7, 'Glossary of Computer Engineering and Programming Terminology'),
 (8, 'On The Equivalence and Transformation of Program Schemes'),
 (9, 'Proposal for an UNCOL'),
 (10, 'Glossary of Computer Engineering and Programming Terminology'),
 (11,
  'The Problem of Programming Communication with Changing Machines A Proposed Solution-Part 2')]

## Question 2.2

In [31]:
def buildDocumentCollectionRegex(fichier):
    #(\.I ([^\n]+))\n(\.T([^\.]*))?
    f = open(fichier, "r")
    tab = re.findall(r"\.I (\d+)\n(.T([^\.]*))?",
           f.read(),
           re.DOTALL)
    tab = [(int(tab[i][0]), re.sub(r"^[\n ]*","",tab[i][2].replace('\n',' '))[:-1]) for i in range(len(tab))]
    return tab

In [32]:
buildDocumentCollectionRegex("cacmShort-good.txt")

[(1, 'Preliminary Report-International Algebraic Language jgkfldjgfkld'),
 (2, 'Extraction of Roots by Repeated Subtractions for Digital Computers'),
 (3, 'Techniques Department on Matrix Program Schemes'),
 (4, 'Glossary of Computer Engineering and Programming Terminology'),
 (5, 'Two Square-Root Approximations'),
 (6, 'The Use of Computers in Inspection Procedures'),
 (7, 'Glossary of Computer Engineering and Programming Terminology'),
 (8, 'On The Equivalence and Transformation of Program Schemes'),
 (9, 'Proposal for an UNCOL'),
 (10, 'Glossary of Computer Engineering and Programming Terminology'),
 (11,
  'The Problem of Programming Communication with Changing Machines A Proposed Solution-Part 2')]

# EXERCICE 3

In [38]:
assert len(buildDocumentCollectionRegex("cisi/cisi.txt")) == 2460
assert len(buildDocumentCollectionRegex("cisi/cisi.qry")) == 112
assert len(buildDocumentCollectionRegex("cacm/cacm.txt")) == 4204
assert len(buildDocumentCollectionRegex("cacm/cacm.qry")) == 64

## Question 3.1

In [39]:
class Document():
    def __init__(self, identifiant, text):
        self.identifiant = identifiant
        self.text = text
        
    def show(self):
        print(self.identifiant, self.text)

## Question 3.2

In [40]:
class Parser():
    def __init__(self, fileURI):
        self.collection = []
        tab = buildDocumentCollectionRegex(fileURI)
        for doc in tab:
            self.collection.append(Document(doc[0], doc[1]))
        

In [41]:
coll = Parser("cisi/cisi.txt")
for d in coll.collection:
    d.show()

1 18 Editions of the Dewey Decimal Classifications
2 Use Made of Technical Libraries
3 Two Kinds of Power An Essay on Bibliographic Control
4 Systems Analysis of a University Library;  final report and research project
5 A Library Management Game: a report on a research project
6 Abstracting Concepts and Methods
7 Academic Library Buildings A Guide to Architectural Issues and Solutions
8 The Academic Library Essays in Honor of Guy 
9 Access to Libraries in College
10 Access to Periodical Resources
11 The Acquisition of Library Materials
12 Acquisition from the 3rd World
13 AD695049
14 The Administration of the College Library
15 Information Flow in Research and Development Laboratories
16 Adopting the Library of Congress Classification System
17 Adventures in Librarianship
18 Selective Dissemination of Information
19 Adventures in Theory of Languages
20 The Age of Jewett: Charles Coffin Jewett and American Librarianship 1841-1868
21 On Aggression
22 Education and Manpower for Librarian

1859 continu fame linkag keen tag 
1860 calderon disclosur lot careless commonli 
1861 haro deliv grotzing mainten conjug 
1862 etr nonsubst extend tip acquir 
1863 constraint panova off sigma tractabl 
1864 upham employ 250 schiminovich iron 
1865 bypass seeker intact girl informatolog 
1866 standard pace moscow supposedli naranan 
1867 promis 22 hindsight wellcom reshelv 
1868 hast interdepartment delhi scholarship solla 
1869 cagan restrict ursinu royal forev 
1870 fortnight reactant 357 infinit interf 
1871 indirectli declin quillian intergr worri 
1872 unchain cooper allow modular ugc 
1873 1965 chauvenet front marczewski recoveri 
1874 gennaro london arthur germani foreign 
1875 microimag test massachusett bundi linear 
1876 memorandum word powerfulli obviou ivori 
1877 tug collin reassert atom technolog 
1878 136 commonwealth becam elm explicitli 
1879 reiter 318 concret proce vander 
1880 come cure josephin jeopard anticanc 
1881 intermedi imperfect protest morozova secretariat